# Notebook 04: ARCH and GARCH Model Estimation

**Objective:**  
- Estimate ARCH and GARCH models for daily returns  
- Interpret volatility parameters economically  
- Examine volatility persistence  

**Asset:** NIFTY 50 Index  
**Frequency:** Daily  

In [2]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 20.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

from arch import arch_model


In [5]:
ticker = "^NSEI"

df = yf.download(
    ticker,
    start="2015-01-01",
    end="2024-12-31",
    progress=False
)

if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

df = df[["Close"]].rename(columns={"Close": "Price"})
df["log_return"] = np.log(df["Price"] / df["Price"].shift(1))
returns = df["log_return"].dropna()


/tmp/ipython-input-199934674.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(


In [13]:
returns_scaled = 100 * returns

In [14]:
arch_1 = arch_model(
    returns_scaled,
    mean="Constant",
    vol="ARCH",
    p=1,
    dist="normal"
)

arch_1_res = arch_1.fit(disp="off")
print(arch_1_res.summary())

                      Constant Mean - ARCH Model Results                      
Dep. Variable:             log_return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -3392.78
Distribution:                  Normal   AIC:                           6791.57
Method:            Maximum Likelihood   BIC:                           6808.99
                                        No. Observations:                 2457
Date:                Sat, Dec 20 2025   Df Residuals:                     2456
Time:                        11:39:18   Df Model:                            1
                                Mean Model                                
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0636  2.143e-02      2.965  3.024e-03 [2.155e-0

In [15]:
garch_11 = arch_model(
    returns_scaled,
    mean="Constant",
    vol="GARCH",
    p=1,
    q=1,
    dist="normal"
)

garch_11_res = garch_11.fit(disp="off")
print(garch_11_res.summary())


                     Constant Mean - GARCH Model Results                      
Dep. Variable:             log_return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -3192.39
Distribution:                  Normal   AIC:                           6392.78
Method:            Maximum Likelihood   BIC:                           6416.01
                                        No. Observations:                 2457
Date:                Sat, Dec 20 2025   Df Residuals:                     2456
Time:                        11:39:27   Df Model:                            1
                                Mean Model                                
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0721  1.759e-02      4.101  4.107e-05 [3.766e-0

In [16]:
params = garch_11_res.params
params

,params
mu,0.072132
omega,0.029765
alpha[1],0.109956
beta[1],0.860927


In [17]:
alpha = params["alpha[1]"]
beta = params["beta[1]"]

alpha + beta

np.float64(0.9708832859612617)

In [18]:
print("ARCH AIC:", arch_1_res.aic)
print("GARCH AIC:", garch_11_res.aic)

ARCH AIC: 6791.5656716471
GARCH AIC: 6392.780207221125


### Interpretation

Both ARCH(1) and GARCH(1,1) models confirm the presence of conditional
heteroskedasticity. The GARCH(1,1) model exhibits strong volatility persistence,
with α + β close to unity, indicating that volatility shocks decay slowly over
time. Compared to ARCH, GARCH provides a more parsimonious and empirically robust
representation of volatility dynamics.